In [1]:
import numpy as np
import pandas as pd
import os
from ipywidgets import IntProgress
from matplotlib import pyplot as plt

In [2]:
contests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['MaxNumberPlayers']]

In [3]:
dirList = os.listdir("data/Chunks/")
fList = []
for filename in dirList: #limit to .csv
    if(filename[-4:] == '.csv'):
        fList.append(filename[:]) 

In [18]:
f = IntProgress(min=0, max=len(fList))
print('Chunks:')
display(f)
importantColumns = ['ContestId', 'EntriesScaled']
for fileName in fList[:]:
    data = pd.read_csv('data/Chunks/'+fileName).drop(columns=['Unnamed: 0'])
    curContests = contests[contests.index.isin(data.ContestId.unique())]
    data = pd.merge(data, curContests, on="ContestId", how='left')
    data = data.sort_values(by='MinutesRemaining', ascending=False)
    data['SummedEntries'] = data.groupby('ContestId')['Entries'].transform(pd.Series.cumsum)
    data['EntriesScaled'] = data['SummedEntries'] / data['MaxNumberPlayers']
    maxTimes = pd.DataFrame(data.groupby('ContestId')['MinutesRemaining'].max())
    maxTimes = maxTimes.rename(columns={'MinutesRemaining':'MaxTime'})
    data = pd.merge(data, maxTimes, on='ContestId', how='left')
    data['TimeScaled'] = (data['MaxTime'] - data['MinutesRemaining']) / data['MaxTime']
    data['Before4HoursOut'] = data['MinutesRemaining'] >= 240
    chunkDF = data[['ContestId', 'EntriesScaled', 'TimeScaled', 'Before4HoursOut']].set_index('ContestId')
    chunkDF = chunkDF.assign(EntriesScaled = chunkDF['EntriesScaled']*100)
    chunkDF = chunkDF.assign(TimeScaled = chunkDF['TimeScaled']*100)
    chunkDF.dropna().to_csv('data/Chunks_Scaled/'+fileName)
    f.value+=1

Chunks:


IntProgress(value=0, max=65)